In [39]:
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, Table, MetaData

# Reflect Tables into SQLAlchemy ORM

In [45]:
# create engine to min_wage.db
engine = create_engine(r'sqlite:///min_wage.sqlite')

# reflect the tables
meta = MetaData()
# meta.reflect(bind = engine)

Base = automap_base(metadata=meta)

Base.prepare(engine, reflect=True)
inspector = inspect(engine)

print(inspector.get_table_names())



# Save references to each table
wages_tbl = Table('wages', meta, autoload=engine)
pce_tbl = Table('spce', meta, autoload=engine)
rpp_tbl = Table('rpp', meta, autoload=engine)


columns = inspector.get_columns('spce')
for c in columns:
    print(c['name'])
pce_tbl

['pce', 'rpp', 'spce', 'states', 'wages', 'years']
Year
state_name
description
pce_value


Table('spce', MetaData(), Column('Year', BIGINT(), table=<spce>), Column('state_name', TEXT(), table=<spce>), Column('description', TEXT(), table=<spce>), Column('pce_value', BIGINT(), table=<spce>), schema=None)

In [25]:
# Create session (link) from Python to the DB
session = Session(bind=engine)

In [33]:
pce_data = session.query(pce_tbl.c.Year,pce_tbl.c.state_name,pce_tbl.c.pce_cat,pce_tbl.c.pce_value)
df = pd.DataFrame(pce_data)
df

AttributeError: Year

# Close Session

In [46]:
# Close Session
session.close()